# Calculate GWP and create tables

The script calculates the GWP of hydrogen and methane using a steady-state perturbation approach.

**Simulations**:
- **CTRL**: fixed surface concentration of hydrogen and methane
- **10H2**: as CTRL, but with surface H2 increased by 10%. Run to steady state.
- **10CH4**: as CTRL, but with surface CH4 increased by 10%. Run to steady state.

In addition, two models have perturbed H2 emissions instead of the surface mole fraction of H2.

GWP is the ratio of the absolute global warming potential (AGWP) for hydrogen to that for CO2. AGWP is defined as the time-integrated radiative forcing to a 1 kg pulse emission of that gas over a given time horizon, here 100 years. 

AGWP is equal to the steady-state radiative forcing (W m-2) divided by the delta flux (Tg-H2 yr-1) for a perturbed vs control. This delta flux between the perturbed (10H2 or 10CH4) and the control simulation (CTRL) includes chemical feedbacks. 

Based on the three set of simulations, we calculate the radiative forcing in the perturbed relative to the control. As the models run with fixed surface concentration of methane, we need to have a separate perturbed methane run. We calculate the radiative forcing per Tg-CH4 (including the feedbacks, as feedbacks are included in the delta flux) for the methane perturbed run, and map the changes in the methane loss in the hydrogen perturbation with the results from the methane perturbed run. 

*GFDL-emi simulations from Paulot et al. (2021) with H2 emissions (200 Tg/yr).*

#### OUTLINE:
**PART I: Read model results**

**PART II: GWP calculations**

**PART III: Main results and tabels**

**Appendix with additional results**

**Uncertainty analysis output**

This is the main notebook for the GWP calculations. The uncertainty analysis and plotting are done in separate notebooks. 

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '{:,.4f}'.format(x) if abs(x)<0 else ('{:,.2f}'.format(x) if abs(x)<10 else ('{:,.1f}'.format(x) if abs(x)<100 else '{:,.0f}'.format(x))))

Input and output path:

In [2]:
path = r"./input/"
outputpath= r"./output/"

Constants:

In [3]:
#AGWP100_CO2 [mW yr m-2 Tg-1] Source: Table 7.SM.6 in IPCC AR6: 0.0895 pW m-2 yr kg-1 (p=10^-12) 
agwp100_CO2 = 0.0895


#CH4 tau_strat[yr] 
tau_strat = 120.0*2.0 #Multiplied by to for not double counting OH loss in stratosphere. 

#CH4 tau_soil [yr] 
tau_soil = 160.0

#Specific RF for CH4 [mW m-2 ppb-1] Etminan et al., 2016
spec_rf_ch4 = 0.44800

#The adjustment is –14% ± 15% IPCC AR6
spec_rf_ch4 = spec_rf_ch4*(1.0-0.14)
print(spec_rf_ch4)

0.38528


**Dry deposition adjustments:** As there is large uncertanty in dry deposition, here is a possibility to spesify the soil sink value. The perturbations are adjusted by the same relative factors as in the control. 

In [4]:
adjust_drydep = False
if(adjust_drydep):
    drydep = 59.0
    outputpath = outputpath + 'drydep_'+ f'{drydep:.0f}_'

# Part I: Read model results

In this part, model results are read from the input files.

For GFDL-emi, 10H2 is the hydrogen only perturbed run, while 10CH4 is the hydrogen plus methane perturbed run.

## 1. Hydrogen budget

### 1.1 H2 burden [Tg]:

In [5]:
file = 'H2_burden.txt'
df_h2_burden = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_h2_burden.loc['10H2']-df_h2_burden.loc['CTRL']
delta.name = 'deltaH2'
df_h2_burden_conc = df_h2_burden.append(delta)
df_h2_burden_conc = df_h2_burden['OSLOCTM3']
df_h2_burden_conc

Scenario
CTRL    195
10H2    nan
10CH4   196
Name: OSLOCTM3, dtype: float64

In [6]:
file = 'OSLOCTM3-emi_burden-H2.csv'
df_h2_burden = pd.read_csv(path + file, sep=',',index_col=0,header=0)
#delta = df_h2_burden.loc['10H2']-df_h2_burden.loc['CTRL']
#delta.name = 'deltaH2'
df_h2_burden.loc['deltaH2'] = df_h2_burden.iloc[0]-df_h2_burden['CNTR'].iloc[0]
df_h2_burden
 

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,205,205,207,226,423,207,207,207,206,207,207,207
deltaH2,0.00,0.22,2.15,21.5,218,2.53,1.93,2.05,1.76,2.56,2.07,1.97


### 1.2 H2 loss
Hydrogen loss happens through two main processes. The largest loss is through soil sink. Remaining hydrogen is lost through reactions with OH as it ascends through the atmosphere. 

#### H2 dry deposition [Tg/yr]

The models diagnose soil sink based on their own schemes. Note that the dry deposition do not affect the atmospheric composition in the concentration driven runs, as the surface concentration of H2 are fixed in these runs.

In [7]:
file = 'H2_drydep.txt'
df_h2_drydep_conc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_drydep_conc = df_h2_drydep_conc['OSLOCTM3']
df_h2_drydep_conc

Scenario
CTRL    59.5
10H2    0.00
10CH4   59.5
Name: OSLOCTM3, dtype: float64

In [8]:
file = 'OSLOCTM3-emi_prod-loss-H2_all_YR13.csv'
df_h2_prodloss = pd.read_csv(path + file, sep=',',index_col=0,header=0)
df_h2_drydep = df_h2_prodloss.loc['Drydeposition']
df_h2_drydep

CNTR       58.0
antro01    58.1
antro1     58.7
antro10    65.0
antro100    128
nemo       58.7
epia       58.7
munich     58.7
usdrydep   58.8
maud       58.7
zep        58.7
maxdep     58.7
Name: Drydeposition, dtype: float64

As drydeposition is uncertain, we can replace the models diagnosed drydeposition by a given value. In the concentration driven run, the drydeposition scheme do not impact the atmospheric composition. For emission driven runs, we also do adjust the drydeposition. Note that we do not use the emission numbers from the emission driven runs directly, the fluxes is estimated based on burden and lifetimes.

In [9]:
if(adjust_drydep):
    print('NB drydep adjusted')
    #
    ##Adjust by the relative adjustment in the control simulations
    #adjust = drydep/df_h2_drydep.loc['CTRL']
    #df_h2_drydep = df_h2_drydep*adjust
    
    #print(df_h2_drydep)

#### H2 atmospheric loss [Tg/yr]

In [10]:
file = 'H2_atm_loss.txt'
df_h2_atmloss_conc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_atmloss_conc = df_h2_atmloss_conc['OSLOCTM3']
df_h2_atmloss_conc


Scenario
CTRL    27.9
10H2    0.00
10CH4   27.2
Name: OSLOCTM3, dtype: float64

In [11]:
df_h2_atmloss = df_h2_prodloss.loc['Total loss'] - df_h2_prodloss.loc['Drydeposition']
df_h2_atmloss

CNTR       29.2
antro01    29.2
antro1     29.5
antro10    32.1
antro100   58.0
nemo       29.5
epia       29.4
munich     29.5
usdrydep   29.4
maud       29.5
zep        29.4
maxdep     29.4
dtype: float64

#### H2 total loss [Tg/yr]:

In [12]:
df_h2_loss_conc = df_h2_atmloss_conc + df_h2_drydep_conc
df_h2_loss_conc


Scenario
CTRL    87.4
10H2    0.00
10CH4   86.7
Name: OSLOCTM3, dtype: float64

In [13]:
df_h2_loss = df_h2_atmloss+ df_h2_drydep
df_h2_loss


CNTR       87.2
antro01    87.3
antro1     88.2
antro10    97.1
antro100    186
nemo       88.2
epia       88.2
munich     88.2
usdrydep   88.2
maud       88.2
zep        88.2
maxdep     88.2
dtype: float64

### 1.3 H2 production

#### H2 atm. prod [Tg/yr]

In [14]:
file = 'H2_atm_prod.txt'
df_h2_atmprod_conc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_atmprod_conc = df_h2_atmprod_conc['OSLOCTM3']
df_h2_atmprod_conc

Scenario
CTRL    55.9
10H2    0.00
10CH4   58.7
Name: OSLOCTM3, dtype: float64

In [15]:
file = 'OSLOCTM3-emi_emissions_H2.csv'
df_h2_emis = pd.read_csv(path + file, sep=',',index_col=0,header=0)
df_h2_emis

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Total emission [Tg],32.2,32.3,33.2,42.3,132,33.2,33.2,33.2,33.2,33.2,33.2,33.2


In [16]:
df_h2_atmprod = df_h2_prodloss.loc['Total production'] - df_h2_emis.iloc[0]
df_h2_atmprod

CNTR       55.8
antro01    55.8
antro1     55.8
antro10    55.7
antro100   54.9
nemo       55.8
epia       55.8
munich     55.8
usdrydep   55.8
maud       55.8
zep        55.8
maxdep     55.8
dtype: float64

### 1.4 Estimated H2 emissions (Total loss = Total prod + emission)

In the runs with fixed surface boundary conditions for H2, there are two unknowns; emissions and soil sink. The models do calculate the soil sink based on their own dry deposition schemes, but it does not influence the H2 in the atmosphere.  
Emission driven runs, are driven by emission estimates and use the dry deposition scheme to calculate the concentration at the surface. At steady state, the total production and total loss must balance:

In [17]:
df_h2_estemis_conc = df_h2_atmloss_conc + df_h2_drydep_conc - df_h2_atmprod_conc
df_h2_estemis_conc

Scenario
CTRL    31.5
10H2    0.00
10CH4   28.0
Name: OSLOCTM3, dtype: float64

In [18]:
df_h2_estemis_conc.loc['10H2']-df_h2_estemis_conc.loc['CTRL']

-31.526913320626726

In [19]:
df_h2_estemis = df_h2_atmloss + df_h2_drydep - df_h2_atmprod
df_h2_estemis

CNTR       31.4
antro01    31.5
antro1     32.4
antro10    41.4
antro100    131
nemo       32.4
epia       32.4
munich     32.4
usdrydep   32.4
maud       32.4
zep        32.4
maxdep     32.4
dtype: float64

In [20]:
df_h2_emis

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Total emission [Tg],32.2,32.3,33.2,42.3,132,33.2,33.2,33.2,33.2,33.2,33.2,33.2


In [21]:
#With more years these should be more similar..

For the emission driven runs, the emissions calculated based on loss terms are similar to the emissions used in these runs.

### 1.5 H2 surface concentration [ppb]

In [22]:
file = 'H2_surfconc.txt'
df_h2_surfconc_conc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_h2_surfconc_conc.loc['10H2']-df_h2_surfconc_conc.loc['CTRL']
delta.name = 'deltaH2'
df_h2_surfconc_conc = df_h2_surfconc_conc.append(delta)
df_h2_surfconc_conc = df_h2_surfconc_conc['OSLOCTM3']
df_h2_surfconc_conc

Scenario
CTRL      532
10H2      nan
deltaH2   nan
Name: OSLOCTM3, dtype: float64

In [23]:
file = 'OSLOCTM3-emi_surfconc-H2.csv'
df_h2_surfconc = pd.read_csv(path + file, sep=',',index_col=0,header=0)
df_h2_surfconc.loc['delta'] = df_h2_surfconc.iloc[0]-df_h2_surfconc['CNTR'].iloc[0]
df_h2_surfconc
#delta = df_h2_surfconc_conc.loc['10H2']-df_h2_surfconc_conc.loc['CTRL']

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
YR13,559,560,565,621,"1,189",567,565,565,564,568,566,565
delta,0.00,0.62,6.22,62.2,630,7.85,6.01,6.23,5.15,9.15,6.63,5.42


### 1.6 H2 lifetime [yr]
We calculate the lifetime as burden divided by loss.

#### H2 total lifetime [yr]

In [24]:
df_h2_lifetime_conc = df_h2_burden_conc/df_h2_loss_conc
df_h2_lifetime_conc


Scenario
CTRL    2.23
10H2     nan
10CH4   2.26
Name: OSLOCTM3, dtype: float64

In [25]:
df_h2_lifetime = df_h2_burden.iloc[0]/df_h2_loss
df_h2_lifetime

CNTR       2.35
antro01    2.35
antro1     2.34
antro10    2.33
antro100   2.27
nemo       2.35
epia       2.34
munich     2.34
usdrydep   2.34
maud       2.35
zep        2.34
maxdep     2.34
dtype: float64

#### H2 atmospheric lifetime [yr]
The atmospheric lifetime is the burden divided by only the atmospheric loss. This is the lifetime of the fraction of hydrogen which is not dry deposited.

In [26]:
df_h2_atm_lifetime_conc = df_h2_burden_conc/df_h2_atmloss_conc
df_h2_atm_lifetime_conc

Scenario
CTRL    6.99
10H2     nan
10CH4   7.20
Name: OSLOCTM3, dtype: float64

In [27]:
df_h2_atm_lifetime = df_h2_burden.iloc[0]/df_h2_atmloss
df_h2_atm_lifetime

CNTR       7.02
antro01    7.02
antro1     7.02
antro10    7.04
antro100   7.28
nemo       7.02
epia       7.02
munich     7.02
usdrydep   7.02
maud       7.02
zep        7.02
maxdep     7.02
dtype: float64

#### H2 soil sink lifetime [yr]

In [28]:
df_h2_soil_sink_lifetime_conc = df_h2_burden_conc/df_h2_drydep_conc
df_h2_soil_sink_lifetime_conc

Scenario
CTRL    3.28
10H2     nan
10CH4   3.29
Name: OSLOCTM3, dtype: float64

In [29]:
df_h2_soil_sink_lifetime = df_h2_burden.iloc[0]/df_h2_drydep
df_h2_soil_sink_lifetime

CNTR       3.53
antro01    3.53
antro1     3.52
antro10    3.48
antro100   3.30
nemo       3.53
epia       3.52
munich     3.52
usdrydep   3.51
maud       3.53
zep        3.52
maxdep     3.52
dtype: float64

### 1.7 H2 flux  [Tg/yr]

The hydrogen flux is calculated as the burden divided by the total hydrogen lifetime. 

The hydrogen lifetime is calculated as burden divided by the total loss (soil sink + atm.loss). 

In stedy state, this is equal to the total loss (se check below)

The difference in calculated flux in the perturbed and control run is calculated. These numbers include feedbacks. 

For the GWP calculations, the radiative forcing in the steady state simulations are divided by these flux numbers.

In [30]:
df_h2_flux_conc = df_h2_burden_conc/df_h2_lifetime_conc
#Add delta flux 10H2:
delta = df_h2_flux_conc.loc['10H2']-df_h2_flux_conc.loc['CTRL']
#delta.name = 'deltaH2'
#df_h2_flux_conc = df_h2_flux_conc.append(delta)
df_h2_flux_conc['deltaH2'] = delta

#Add delta flux 10CH4:
delta = df_h2_flux_conc.loc['10CH4']-df_h2_flux_conc.loc['CTRL']
#delta.name = 'deltaCH4'
#df_h2_flux_conc = df_h2_flux_conc.append(delta)
df_h2_flux_conc['deltaCH4'] = delta
df_h2_flux_conc


Scenario
CTRL        87.4
10H2         nan
10CH4       86.7
deltaH2      nan
deltaCH4   -0.75
Name: OSLOCTM3, dtype: float64

In [31]:
df_h2_flux = df_h2_burden.iloc[0]/df_h2_lifetime
df_h2_flux.name = 'h2_flux'

delta = df_h2_flux-df_h2_flux['CNTR']
delta.name = 'deltaH2'

df_h2_flux =pd.concat([df_h2_flux,delta],axis=1)

df_h2_flux['deltaCH4'] = df_h2_flux_conc['deltaCH4']
df_h2_flux
##Add delta flux 10CH4:
#delta = df_h2_flux.loc['10CH4']-df_h2_flux.loc['CTRL']
##delta.name = 'deltaCH4'
##df_h2_flux_conc = df_h2_flux_conc.append(delta)
#df_h2_flux_conc['deltaCH4'] = delta
#df_h2_flux_conc

,h2_flux,deltaH2,deltaCH4
CNTR,87.2,0.00,-0.75
antro01,87.3,0.10,-0.75
antro1,88.2,0.99,-0.75
antro10,97.1,9.90,-0.75
antro100,186,99.1,-0.75
nemo,88.2,0.99,-0.75
epia,88.2,0.99,-0.75
munich,88.2,0.99,-0.75
usdrydep,88.2,0.99,-0.75
maud,88.2,0.99,-0.75


In the methane run, the hydrogen surface concentration is kept fixed. Enhancing methane would influence H2. The hydrogen concentration would have increased, but since we run with fixed concentration, there is a negative flux to compensate. So the increased flux in H2 due to methane is -1*deltaCH4.

In [32]:
#Check
df_h2_loss_conc

Scenario
CTRL    87.4
10H2    0.00
10CH4   86.7
Name: OSLOCTM3, dtype: float64

In [33]:
df_h2_loss

CNTR       87.2
antro01    87.3
antro1     88.2
antro10    97.1
antro100    186
nemo       88.2
epia       88.2
munich     88.2
usdrydep   88.2
maud       88.2
zep        88.2
maxdep     88.2
dtype: float64

## 2. Methane results

### 2.1 CH4 burden [Tg]

In [34]:
file = 'CH4_burden.txt'
df_ch4_burden_conc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_ch4_burden_conc.loc['10CH4']-df_ch4_burden_conc.loc['CTRL']
delta.name = 'deltaCH4'
df_ch4_burden_conc = df_ch4_burden_conc.append(delta)
df_ch4_burden_conc = df_ch4_burden_conc['OSLOCTM3']
df_ch4_burden_conc

Scenario
CTRL       4,975
10H2         nan
10CH4      5,474
deltaCH4     498
Name: OSLOCTM3, dtype: float64

In [35]:
file = 'OSLOCTM3-emi_burden-CH4.csv'
df_ch4_burden = pd.read_csv(path + file, sep=',',index_col=0,header=0)
df_ch4_burden.loc['deltaCH4'] = df_ch4_burden.iloc[0]-df_ch4_burden['CNTR'].iloc[0]
df_ch4_burden

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,"4,975","4,975","4,975","4,975","4,975","4,975","4,975","4,975","4,975","4,975","4,975","4,975"
deltaCH4,0.00,0.00,0.00,0.05,0.43,0.01,0.00,0.00,0.00,0.01,0.00,0.00


### 2.2 CH4 atmospheric loss [Tg/yr]

This is atmospheric loss due to OH.

In [36]:
file = 'CH4_loss.txt'
df_ch4_loss_conc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_ch4_loss_conc = df_ch4_loss_conc['OSLOCTM3']
df_ch4_loss_conc

Scenario
CTRL    674
10H2    nan
10CH4   719
Name: OSLOCTM3, dtype: float64

In [37]:
file = 'OSLOCTM3-emi_prod-loss-CH4_all_YR13.csv'
df_ch4_prodloss = pd.read_csv(path + file, sep=',',index_col=0,header=0)
df_ch4_loss=df_ch4_prodloss.loc['CH4 + OH -> H2O + CH3']
df_ch4_loss

CNTR       673
antro01    673
antro1     673
antro10    670
antro100   644
nemo       672
epia       673
munich     673
usdrydep   673
maud       672
zep        673
maxdep     673
Name: CH4 + OH -> H2O + CH3, dtype: float64

### 2.3 CH4 surface concentration [ppb]

In [38]:
file = 'CH4_surfconc.txt'
df_ch4_surfconc_conc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_ch4_surfconc_conc.loc['10CH4']-df_ch4_surfconc_conc.loc['CTRL']
delta.name = 'deltaCH4'
df_ch4_surfconc_conc = df_ch4_surfconc_conc.append(delta)
df_ch4_surfconc_conc = df_ch4_surfconc_conc['OSLOCTM3']
df_ch4_surfconc_conc

Scenario
CTRL       1,813
10CH4      1,994
deltaCH4     181
Name: OSLOCTM3, dtype: float64

In [39]:
file = 'OSLOCTM3-emi_surfconc-CH4.csv'
df_ch4_surfconc = pd.read_csv(path + file, sep=',',index_col=0,header=0)
df_ch4_surfconc.loc['delta'] = df_ch4_surfconc.iloc[0]-df_ch4_surfconc['CNTR'].iloc[0]
df_ch4_surfconc.loc['delta'] = df_ch4_surfconc_conc['deltaCH4']
df_ch4_surfconc

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
YR13,"1,813","1,813","1,813","1,813","1,813","1,813","1,813","1,813","1,813","1,813","1,813","1,813"
delta,181,181,181,181,181,181,181,181,181,181,181,181


### 2.4 CH4 lifetime [yr]

#### Lifetime due to OH [yr]

In [40]:
df_ch4_lifetime_conc = df_ch4_burden_conc.drop('deltaCH4')/df_ch4_loss_conc
df_ch4_lifetime_conc

Scenario
CTRL    7.38
10H2     nan
10CH4   7.62
Name: OSLOCTM3, dtype: float64

In [41]:
df_ch4_lifetime = df_ch4_burden.drop('deltaCH4')/df_ch4_loss
df_ch4_lifetime

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,7.39,7.39,7.40,7.43,7.73,7.40,7.40,7.40,7.40,7.40,7.40,7.40


#### Total CH4 lifetime [yr]

Inverse lifetimes (mean loss frequencies) are additive [Forster et al.,2007; Prather, 2007]. Added lifetime due to stratospheric chemistry in addion to OH and soil sink.

In [42]:
df_ch4_tot_lifetime_conc = 1.0/(1.0/df_ch4_lifetime_conc + 1.0/tau_strat + 1.0/tau_soil)
df_ch4_tot_lifetime_conc

Scenario
CTRL    6.85
10H2     nan
10CH4   7.06
Name: OSLOCTM3, dtype: float64

In [43]:
df_ch4_tot_lifetime = 1.0/(1.0/df_ch4_lifetime + 1.0/tau_strat + 1.0/tau_soil)
df_ch4_tot_lifetime

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,6.87,6.87,6.87,6.89,7.15,6.87,6.87,6.87,6.87,6.87,6.87,6.87


#### Other methane loss term, dervided based on the lifetime and burden.

In [44]:
#Soil loss:
df_ch4_loss_soil_conc = df_ch4_burden_conc.drop('deltaCH4')/tau_soil
df_ch4_loss_soil_conc

Scenario
CTRL    31.1
10H2     nan
10CH4   34.2
Name: OSLOCTM3, dtype: float64

In [45]:
df_ch4_loss_soil = df_ch4_burden.drop('deltaCH4')/tau_soil
df_ch4_loss_soil

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,31.1,31.1,31.1,31.1,31.1,31.1,31.1,31.1,31.1,31.1,31.1,31.1


In [46]:
#Strat chemistry loss (other than OH)
df_ch4_loss_other_strat_conc = df_ch4_burden_conc.drop('deltaCH4')/(tau_strat)
df_ch4_loss_other_strat_conc

Scenario
CTRL    20.7
10H2     nan
10CH4   22.8
Name: OSLOCTM3, dtype: float64

In [47]:
df_ch4_loss_other_strat = df_ch4_burden.drop('deltaCH4')/(tau_strat)
df_ch4_loss_other_strat

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,20.7,20.7,20.7,20.7,20.7,20.7,20.7,20.7,20.7,20.7,20.7,20.7


### 2.5 CH4 flux  [Tg/yr]

The methane flux is calculated as the burden divided by the total methane lifetime.

The difference in calculated flux in the perturbed and control run is calculated. These numbers include feedbacks.

In [48]:
df_ch4_flux_conc = df_ch4_burden_conc.drop('deltaCH4')/df_ch4_tot_lifetime_conc
##Add delta CH4 flux 10H2
#delta = df_ch4_flux_conc.loc['10H2']-df_ch4_flux_conc.loc['CTRL']
#delta.name = 'deltaH2'
#df_ch4_flux_conc = df_ch4_flux_conc.append(delta)
#Add delta CH4 flux 10CH4

#delta = df_ch4_flux_conc.loc['10CH4']-df_ch4_flux_conc.loc['CTRL']
#delta.name = 'deltaCH4'

#df_ch4_flux_conc = df_ch4_flux_conc.append(delta)
df_ch4_flux_conc['deltaCH4'] = df_ch4_flux_conc.loc['10CH4']-df_ch4_flux_conc.loc['CTRL']
df_ch4_flux_conc

##Add same flux in OsloCTM3-emi as in OsloCTM3
#df_ch4_flux['OSLOCTM3-emi'].loc['deltaCH4']= df_ch4_flux['OSLOCTM3'].loc['deltaCH4']
#df_ch4_flux


Scenario
CTRL        726
10H2        nan
10CH4       776
deltaCH4   49.7
Name: OSLOCTM3, dtype: float64

In [49]:
df_ch4_flux = df_ch4_burden.drop('deltaCH4')/df_ch4_tot_lifetime
df_ch4_flux.loc['deltaH2'] = df_ch4_flux.iloc[0]-df_ch4_flux['CNTR'].iloc[0]

#Add same ch4 flux in OsloCTM3 sensitivity tests as in OsloCTM3 conc
df_ch4_flux.loc['deltaCH4'] =  df_ch4_flux_conc['deltaCH4']
df_ch4_flux

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,725,725,724,722,696,724,724,724,724,724,724,724
deltaH2,0.00,-0.03,-0.30,-3.03,-28.7,-0.35,-0.27,-0.29,-0.25,-0.35,-0.29,-0.28
deltaCH4,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7


## 3. Ozone burden and RF

### 3.1 Change in tropospheric ozone (DU)

The tropopause definition is the model layer in the control simulation where 150 ppbv ozone are reached. 

Keep in mind that:
for GFDL-emi 10H2 is H2 perturbation (200 Tg/yr) and 10CH4 is H2+CH4 pert. And that for OsloCTM-emi, the same sensitivity for methane perturbation is used as for OsloCTM concentration driven.

In [50]:
file = 'ozone_du_trop.txt'
df_ozone_du_trop_conc = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_du_trop_conc = df_ozone_du_trop_conc['OSLOCTM3']
df_ozone_du_trop_conc

Scenario
10H2     nan
10CH4   0.83
Name: OSLOCTM3, dtype: float64

In [51]:
#ADD here for senstest

In [52]:
#For OsloCTM-emi, use the methane results from the concentration driven methane experiment.
#df_ozone_du_trop['OSLOCTM3-emi'].loc['10CH4'] = df_ozone_du_trop['OSLOCTM3'].loc['10CH4'] 
#df_ozone_du_trop

### 3.2 Change in stratospheric ozone (DU)

In [53]:
file = 'ozone_du_strat.txt'
df_ozone_du_strat_conc = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_du_strat_conc = df_ozone_du_strat_conc['OSLOCTM3']
df_ozone_du_strat_conc

Scenario
10H2     nan
10CH4   0.84
Name: OSLOCTM3, dtype: float64

In [54]:
#ADD here for senstest


In [55]:
#For OsloCTM-emi, use the methane results from the concentration driven methane experiment.
#df_ozone_du_strat['OSLOCTM3-emi'].loc['10CH4'] = df_ozone_du_strat['OSLOCTM3'].loc['10CH4'] 
#df_ozone_du_strat

### 3.5 Ozone RF

Ozone RF is calculated using a radiative kernel (Skeie et al 2020) and the modelled changes in ozone. 

Keep in mind that:
for GFDL-emi 10H2 RF is forcing calculated in the H2 perturbation and 10CH4 the forcing calculated by the H2+CH4.

In [56]:
file = 'ozone_rf.txt'
df_ozone_rf_conc = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_rf_conc = df_ozone_rf_conc['OSLOCTM3'] 
df_ozone_rf_conc 

Scenario
10H2     nan
10CH4   40.8
Name: OSLOCTM3, dtype: float64

In [57]:
#ADD here for senstest
df_ozone_rf = pd.DataFrame(data=[],columns=df_ch4_flux.columns,index=df_ozone_rf_conc.index)
df_ozone_rf

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
10H2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10CH4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
file = 'RF_NRFmethod_OsloCTM3_senstest_yr26_net_yearly.csv'
ozone_rf_orig = pd.read_csv(path+file,sep=';',index_col=0,header=0,skiprows=3)
ozone_rf_orig['NET adj.']


CNTR (mW m-2)             nan
antro01-CNTR (mW m-2)    0.02
antro1-CNTR (mW m-2)     0.21
antro10-CNTR (mW m-2)    2.14
antro100-CNTR (mW m-2)   21.4
epia-CNTR (mW m-2)       0.18
nemo-CNTR (mW m-2)       0.23
munich-CNTR (mW m-2)     0.20
usdrydep-CNTR (mW m-2)   0.17
zep-CNTR (mW m-2)        0.19
maud-CNTR (mW m-2)       0.23
maxdep-CNTR (mW m-2)     0.20
Name: NET adj., dtype: float64

In [59]:
for scen in df_ozone_rf.columns[1:]:
    df_ozone_rf.loc['10H2'][scen] = ozone_rf_orig['NET adj.'].loc[scen+'-CNTR (mW m-2)']
df_ozone_rf
    


,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
10H2,NaN,0.02,0.21,2.14,21.4,0.23,0.18,0.20,0.17,0.23,0.19,0.20
10CH4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
#In OsloCTM-emi use the same RF in the methane pertubation as in OsloCTM3
df_ozone_rf.loc['10CH4'] = df_ozone_rf_conc.loc['10CH4'] 
df_ozone_rf

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
10H2,NaN,0.02,0.21,2.14,21.4,0.23,0.18,0.20,0.17,0.23,0.19,0.20
10CH4,40.8,40.8,40.8,40.8,40.8,40.8,40.8,40.8,40.8,40.8,40.8,40.8


## 4. Stratospheric H2O RF [mW m-2]

Stratospheric H2O RF calculated offline.

In [61]:
file = 'H2O_rf.txt'
df_h2o_rf_conc = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_h2o_rf_conc = df_h2o_rf_conc['OSLOCTM3']
df_h2o_rf_conc

Scenario
10H2     nan
10CH4   9.53
Name: OSLOCTM3, dtype: float64

In [62]:
#ADD here for senstest
df_h2o_rf = pd.DataFrame(data=[],columns=df_ch4_flux.columns,index=df_h2o_rf_conc.index)
df_h2o_rf

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
10H2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10CH4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
file = 'strat_h2o_h2.csv'
df_h2o_rf_orig = pd.read_csv(path+file,sep='\t',index_col=0,header=0)
df_h2o_rf_orig


,an01,ant1,an10,a100,zepl,maud,nemo,epia,munc,usdd,maxd
NET,0.02,0.17,1.67,16.8,0.15,0.18,0.18,0.14,0.15,0.13,0.16


In [64]:
scen_dict_strat_h2o = {'an01':'antro01',
                       'ant1':'antro1',
                       'an10':'antro10',
                       'a100':'antro100',
                       'zepl':'zep',
                       'maud':'maud',
                       'nemo':'nemo',
                       'epia':'epia',
                       'munc':'munich',
                       'usdd':'usdrydep',
                       'maxd':'maxdep'}
for scen in scen_dict_strat_h2o:
    df_h2o_rf[scen_dict_strat_h2o[scen]].loc['10H2'] = df_h2o_rf_orig[scen].loc['NET']
df_h2o_rf

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
10H2,NaN,0.02,0.17,1.67,16.8,0.18,0.14,0.15,0.13,0.18,0.15,0.16
10CH4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
#In OsloCTM-emi use the same RF in the methane pertubation as in OsloCTM3
df_h2o_rf.loc['10CH4'] = df_h2o_rf_conc.loc['10CH4'] 
df_h2o_rf

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
10H2,NaN,0.02,0.17,1.67,16.8,0.18,0.14,0.15,0.13,0.18,0.15,0.16
10CH4,9.53,9.53,9.53,9.53,9.53,9.53,9.53,9.53,9.53,9.53,9.53,9.53


## 5. Aerosol RF [mW m-2]

In [66]:
file = 'aerosol_rf.txt'
df_aerosol_rf_conc = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_aerosol_rf_conc = df_aerosol_rf_conc['OSLOCTM3']
df_aerosol_rf_conc

Scenario
10H2    nan
10CH4   nan
Name: OSLOCTM3, dtype: float64

In [67]:
#ADD here for senstest
df_aerosol_rf = pd.DataFrame(data=[],columns=df_ch4_flux.columns,index=df_h2o_rf_conc.index)
df_aerosol_rf

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
10H2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10CH4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
#In OsloCTM-emi use the same RF in the methane pertubation as in OsloCTM3
df_aerosol_rf.loc['10CH4'] = df_aerosol_rf_conc.loc['10CH4'] 
df_aerosol_rf

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
10H2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10CH4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



# Part II: GWP calculations:

The pulse integrated to infinity of the effects of a short lived climate forcer is equal to the change respones of its effects at steady state multiplied by the steady state lifetime of the short lived forcer(:cite:p:`Prather2002a` and :cite:p:`Prather2007a`). 

Prather 2002: prove that: (a) the steadystate pattern of impacts caused by specified emissions, multiplied by (b) the steady-state lifetime of the source gas for that emission pattern, is exactly equal to (c) the integral of all impacts - independent of the number and atmospheric residence times of secondary impacts. Therefore, the AGWP for hydrogen is identical whether calculating by integrating a pulse or by using the steady state changes per flux, given that the perturbation reaches steady state before 100 years. The longest-lived chemical mode here is keyed to methane, which has an e-folding lifetime of about 12 years. Our perturbed experiments are run to steate-state.


### Change in H2 surface conc. caused by 1 Tg H2/yr [ppb yr Tg-1]

This is not used for the GWP calculation. Only for the per flux table and for the feedback factor calulations.

In [69]:
df_ch4_flux

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,725,725,724,722,696,724,724,724,724,724,724,724
deltaH2,0.00,-0.03,-0.30,-3.03,-28.7,-0.35,-0.27,-0.29,-0.25,-0.35,-0.29,-0.28
deltaCH4,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7


In [70]:
df_surf_h2_per_h2_flux = df_h2_surfconc.loc['delta']/df_h2_flux['deltaH2']
df_surf_h2_per_h2_flux.name = 'surf_h2_per_h2_flux'
df_surf_h2_per_h2_flux

CNTR        nan
antro01    6.27
antro1     6.27
antro10    6.28
antro100   6.35
nemo       7.92
epia       6.05
munich     6.32
usdrydep   5.20
maud       9.24
zep        6.69
maxdep     5.48
Name: surf_h2_per_h2_flux, dtype: float64

### Change in CH4 flux caused by 1 TgH2 /yr (includes H2 feedback) [Tg CH4/Tg H2]:

The ch4_flux is multiplied by -1 (see above).

In [71]:
df_ch4_flux_per_h2_flux = -1.0*df_ch4_flux.loc['deltaH2']/df_h2_flux['deltaH2']
df_ch4_flux_per_h2_flux.name = 'ch4_flux_per_h2_flux'
df_ch4_flux_per_h2_flux

CNTR        nan
antro01    0.30
antro1     0.31
antro10    0.31
antro100   0.29
nemo       0.35
epia       0.28
munich     0.29
usdrydep   0.25
maud       0.35
zep        0.29
maxdep     0.28
Name: ch4_flux_per_h2_flux, dtype: float64

### Change in CH4 surface conc. caused by 1 Tg/yr CH4 [ppb yr/Tg CH4]

In [72]:
df_surf_ch4_per_ch4_flux_conc =  df_ch4_surfconc_conc.loc['deltaCH4']/df_ch4_flux_conc.loc['deltaCH4']
df_surf_ch4_per_ch4_flux_conc


3.650302696662327

In [73]:
df_surf_ch4_per_ch4_flux =  df_ch4_surfconc.loc['delta']/df_ch4_flux.loc['deltaCH4']
df_surf_ch4_per_ch4_flux.name = 'surf_ch4_per_ch4_flux'
df_surf_ch4_per_ch4_flux

CNTR       3.65
antro01    3.65
antro1     3.65
antro10    3.65
antro100   3.65
nemo       3.65
epia       3.65
munich     3.65
usdrydep   3.65
maud       3.65
zep        3.65
maxdep     3.65
Name: surf_ch4_per_ch4_flux, dtype: float64

OsloCTM3-emi set equal to OsloCTM3 concentration driven.

In [74]:
df_surf_ch4_per_ch4_flux[:] = df_surf_ch4_per_ch4_flux_conc
df_surf_ch4_per_ch4_flux

CNTR       3.65
antro01    3.65
antro1     3.65
antro10    3.65
antro100   3.65
nemo       3.65
epia       3.65
munich     3.65
usdrydep   3.65
maud       3.65
zep        3.65
maxdep     3.65
Name: surf_ch4_per_ch4_flux, dtype: float64

### Change in CH4 surface concentration per emission H2 [ppb yr /Tg H2]

In [75]:
df_surf_ch4_per_h2_flux = df_surf_ch4_per_ch4_flux*df_ch4_flux_per_h2_flux
df_surf_ch4_per_h2_flux.name = 'surf_ch4_per_h2_flux'
df_surf_ch4_per_h2_flux

CNTR        nan
antro01    1.11
antro1     1.12
antro10    1.12
antro100   1.06
nemo       1.29
epia       1.01
munich     1.08
usdrydep   0.92
maud       1.29
zep        1.06
maxdep     1.03
Name: surf_ch4_per_h2_flux, dtype: float64

In [76]:
df_h2_flux

,h2_flux,deltaH2,deltaCH4
CNTR,87.2,0.00,-0.75
antro01,87.3,0.10,-0.75
antro1,88.2,0.99,-0.75
antro10,97.1,9.90,-0.75
antro100,186,99.1,-0.75
nemo,88.2,0.99,-0.75
epia,88.2,0.99,-0.75
munich,88.2,0.99,-0.75
usdrydep,88.2,0.99,-0.75
maud,88.2,0.99,-0.75


### Change in H2 flux caused by 1 TgCH4/yr [Tg H2/Tg CH4]

We multiply by -1 (see above)

In [77]:
df_h2_flux_per_ch4_flux = -1.0*df_h2_flux['deltaCH4']/df_ch4_flux.loc['deltaCH4']
df_h2_flux_per_ch4_flux.name = 'h2_flux_per_ch4_flux'
df_h2_flux_per_ch4_flux



CNTR       0.02
antro01    0.02
antro1     0.02
antro10    0.02
antro100   0.02
nemo       0.02
epia       0.02
munich     0.02
usdrydep   0.02
maud       0.02
zep        0.02
maxdep     0.02
Name: h2_flux_per_ch4_flux, dtype: float64

### HYDROGEN AGWP100 CH4 [mW m-2 yr Tg-1]

agwp_ch4 = RF per flux H2

In [78]:
df_h2_agwp_ch4 = df_surf_ch4_per_h2_flux*spec_rf_ch4
df_h2_agwp_ch4.name = 'h2_agwp_ch4'
df_h2_agwp_ch4

CNTR        nan
antro01    0.43
antro1     0.43
antro10    0.43
antro100   0.41
nemo       0.50
epia       0.39
munich     0.41
usdrydep   0.36
maud       0.50
zep        0.41
maxdep     0.40
Name: h2_agwp_ch4, dtype: float64

In [79]:
#agwp_ch4 = RF per flux H2 (For the per flux table)
df_ch4_rf_per_h2_flux = df_surf_ch4_per_h2_flux*spec_rf_ch4
df_ch4_rf_per_h2_flux.name = 'ch4_rf_per_h2_flux'

df_ch4_rf_per_h2_flux

CNTR        nan
antro01    0.43
antro1     0.43
antro10    0.43
antro100   0.41
nemo       0.50
epia       0.39
munich     0.41
usdrydep   0.36
maud       0.50
zep        0.41
maxdep     0.40
Name: ch4_rf_per_h2_flux, dtype: float64

### Initialize H2 GWP table

In [80]:
antmod = len(df_h2_agwp_ch4.index)
df_h2_gwp = pd.DataFrame(np.empty([5,antmod])*np.nan,columns=df_h2_agwp_ch4.index,
                         index=['O3','CH4','strat H2O','O3 CH4ind','strat H2O CH4ind'])
df_h2_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
CH4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
strat H2O,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
O3 CH4ind,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
strat H2O CH4ind,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


### Add methane GWP

In [81]:
df_h2_gwp.loc['CH4'] = df_h2_agwp_ch4/agwp100_CO2
df_h2_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
CH4,nan,4.78,4.82,4.80,4.56,5.56,4.33,4.63,3.97,5.57,4.57,4.42
strat H2O,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
O3 CH4ind,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
strat H2O CH4ind,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


### HYDROGEN AGWP100 strat H2O [mW m-2 yr Tg-1]

In [82]:
df_h2_agwp_h2o = df_h2o_rf.loc['10H2'].astype(float)/df_h2_flux['deltaH2']
df_h2_agwp_h2o

CNTR        nan
antro01    0.17
antro1     0.17
antro10    0.17
antro100   0.17
nemo       0.18
epia       0.14
munich     0.15
usdrydep   0.13
maud       0.18
zep        0.15
maxdep     0.16
dtype: float64

In [83]:
#Add to the flux table
df_h2o_rf_per_h2_flux = df_h2o_rf.loc['10H2'].astype(float)/df_h2_flux['deltaH2']
df_h2o_rf_per_h2_flux.name= 'h2o_rf_per_h2_flux'


#Strat H2O RF per methane flux 
df_h2o_rf_per_ch4_flux = df_h2o_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_h2o_rf_per_ch4_flux.name = 'h2o_rf_per_ch4_flux'


In [84]:
#df_h2o_rf_per_ch4_flux['OSLOCTM3-emi'] = df_h2o_rf_per_ch4_flux['OSLOCTM3']
#df_h2o_rf_per_ch4_flux

### Add stratospheric H2O GWP

In [85]:
df_h2_gwp.loc['strat H2O'] = df_h2_agwp_h2o/agwp100_CO2
df_h2_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
CH4,nan,4.78,4.82,4.80,4.56,5.56,4.33,4.63,3.97,5.57,4.57,4.42
strat H2O,nan,1.87,1.87,1.88,1.90,2.05,1.57,1.69,1.45,2.05,1.65,1.80
O3 CH4ind,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
strat H2O CH4ind,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


### HYDROGEN AGWP100 O3 [mW m-2 yr Tg-1]

In [86]:
df_h2_agwp_o3 = df_ozone_rf.loc['10H2'].astype(float)/df_h2_flux['deltaH2']
df_h2_agwp_o3.name = 'h2_agwp_o3'

df_h2_agwp_o3

CNTR        nan
antro01    0.19
antro1     0.21
antro10    0.22
antro100   0.22
nemo       0.23
epia       0.18
munich     0.20
usdrydep   0.17
maud       0.23
zep        0.19
maxdep     0.20
Name: h2_agwp_o3, dtype: float64

In [87]:
#Similar, but use only the H2 Ozone RF for GFDL. To be used in the table:
df_ozone_rf_per_h2_flux = df_ozone_rf.loc['10H2'].astype(float)/df_h2_flux['deltaH2']
df_ozone_rf_per_h2_flux.name= 'ozone_rf_per_h2_flux'

df_ozone_rf_per_h2_flux


CNTR        nan
antro01    0.19
antro1     0.21
antro10    0.22
antro100   0.22
nemo       0.23
epia       0.18
munich     0.20
usdrydep   0.17
maud       0.23
zep        0.19
maxdep     0.20
Name: ozone_rf_per_h2_flux, dtype: float64

In [88]:
#Ozone RF per methane flux (move to the methane part?)
df_ozone_rf_per_ch4_flux = df_ozone_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_ozone_rf_per_ch4_flux.name = 'ozone_rf_per_ch4_flux'

In [89]:
df_ozone_rf_per_ch4_flux

CNTR       0.82
antro01    0.82
antro1     0.82
antro10    0.82
antro100   0.82
nemo       0.82
epia       0.82
munich     0.82
usdrydep   0.82
maud       0.82
zep        0.82
maxdep     0.82
Name: ozone_rf_per_ch4_flux, dtype: object

### Add Ozone GWP

In [90]:
df_h2_gwp.loc['O3'] = df_h2_agwp_o3/agwp100_CO2
df_h2_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,nan,2.13,2.38,2.42,2.41,2.59,2.06,2.22,1.89,2.62,2.14,2.27
CH4,nan,4.78,4.82,4.80,4.56,5.56,4.33,4.63,3.97,5.57,4.57,4.42
strat H2O,nan,1.87,1.87,1.88,1.90,2.05,1.57,1.69,1.45,2.05,1.65,1.80
O3 CH4ind,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
strat H2O CH4ind,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


### For the per flux table

In [91]:
df_ch4_flux


,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,725,725,724,722,696,724,724,724,724,724,724,724
deltaH2,0.00,-0.03,-0.30,-3.03,-28.7,-0.35,-0.27,-0.29,-0.25,-0.35,-0.29,-0.28
deltaCH4,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7


In [92]:
#df_trop_du_ozone_per_ch4_flux = df_ozone_du_trop.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
#df_trop_du_ozone_per_ch4_flux.name = 'trop_du_ozone_per_ch4_flux'

#df_strat_du_ozone_per_ch4_flux = df_ozone_du_strat.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
#df_strat_du_ozone_per_ch4_flux.name = 'strat_du_ozone_per_ch4_flux'

#df_trop_du_ozone_per_h2_flux = df_ozone_du_trop.loc['10H2']/df_h2_flux['deltaH2']
#df_trop_du_ozone_per_h2_flux.name = 'trop_du_ozone_per_h2_flux'

#df_strat_du_ozone_per_h2_flux = df_ozone_du_strat.loc['10H2']/df_h2_flux['deltaH2']
#df_strat_du_ozone_per_h2_flux.name = 'strat_du_ozone_per_h2_flux'

In [93]:
#df_trop_du_ozone_per_ch4_flux['OSLOCTM3-emi']=df_trop_du_ozone_per_ch4_flux['OSLOCTM3']
#df_strat_du_ozone_per_ch4_flux['OSLOCTM3-emi']=df_strat_du_ozone_per_ch4_flux['OSLOCTM3']

### HYDROGEN AGWP100 aerosol [mW m-2 yr Tg-1]

In [94]:
df_h2_agwp_aerosol = df_aerosol_rf.loc['10H2'].astype(float)/df_h2_flux['deltaH2']
df_h2_agwp_aerosol.name = 'h2_agwp_aerosol'
df_h2_agwp_aerosol
#NBNB GFDL-emi include methane induced.

CNTR       nan
antro01    nan
antro1     nan
antro10    nan
antro100   nan
nemo       nan
epia       nan
munich     nan
usdrydep   nan
maud       nan
zep        nan
maxdep     nan
Name: h2_agwp_aerosol, dtype: float64

In [95]:
df_h2_gwp.loc['aerosol'] = df_h2_agwp_aerosol/agwp100_CO2 

In [96]:
#Add to the flux table
df_aerosol_rf_per_h2_flux = df_aerosol_rf.loc['10H2'].astype(float)/df_h2_flux['deltaH2']
df_aerosol_rf_per_h2_flux.name= 'aerosol_rf_per_h2_flux'


df_aerosol_rf_per_ch4_flux = df_aerosol_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_aerosol_rf_per_ch4_flux.name = 'aerosol_rf_per_ch4_flux'


In [97]:
df_aerosol_rf_per_ch4_flux

CNTR        NaN
antro01     NaN
antro1      NaN
antro10     NaN
antro100    NaN
nemo        NaN
epia        NaN
munich      NaN
usdrydep    NaN
maud        NaN
zep         NaN
maxdep      NaN
Name: aerosol_rf_per_ch4_flux, dtype: object

## Methane induced GWP:

### HYDROGEN AGWP100 methane induced O3 [mW m-2 yr Tg-1]

It does not matter here if we use surface concentration, burden or tropospheric concentration.

In [98]:
df_surf_ch4_per_h2_flux


CNTR        nan
antro01    1.11
antro1     1.12
antro10    1.12
antro100   1.06
nemo       1.29
epia       1.01
munich     1.08
usdrydep   0.92
maud       1.29
zep        1.06
maxdep     1.03
Name: surf_ch4_per_h2_flux, dtype: float64

In [99]:
#Wm-2/ppbCH4*ppbCH4/TgH2yr-1 -> Wm-2/TgH2yr-1
df_h2_agwp_ch4ind_o3 = df_ozone_rf.loc['10CH4'].astype(float)/df_ch4_surfconc.loc['delta']*df_surf_ch4_per_h2_flux
df_h2_agwp_ch4ind_o3.name = 'h2_agwp_ch4ind_o3'
df_h2_agwp_ch4ind_o3

CNTR        nan
antro01    0.25
antro1     0.25
antro10    0.25
antro100   0.24
nemo       0.29
epia       0.23
munich     0.24
usdrydep   0.21
maud       0.29
zep        0.24
maxdep     0.23
Name: h2_agwp_ch4ind_o3, dtype: float64

### Add methane induced O3 GWP

In [100]:
df_h2_gwp.loc['O3 CH4ind'] = df_h2_agwp_ch4ind_o3/agwp100_CO2
df_h2_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,nan,2.13,2.38,2.42,2.41,2.59,2.06,2.22,1.89,2.62,2.14,2.27
CH4,nan,4.78,4.82,4.80,4.56,5.56,4.33,4.63,3.97,5.57,4.57,4.42
strat H2O,nan,1.87,1.87,1.88,1.90,2.05,1.57,1.69,1.45,2.05,1.65,1.80
O3 CH4ind,nan,2.79,2.81,2.80,2.66,3.25,2.53,2.70,2.32,3.25,2.67,2.58
strat H2O CH4ind,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
aerosol,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


### HYDROGEN AGWP100 methane induced strat H2O [mW m-2 yr Tg-1]

In [101]:
df_h2_agwp_ch4ind_h2o = df_h2o_rf.loc['10CH4']/df_ch4_surfconc.loc['delta']*df_surf_ch4_per_h2_flux
df_h2_agwp_ch4ind_h2o.name = 'h2_agwp_ch4ind_h2o'
df_h2_agwp_ch4ind_h2o

CNTR        NaN
antro01    0.06
antro1     0.06
antro10    0.06
antro100   0.06
nemo       0.07
epia       0.05
munich     0.06
usdrydep   0.05
maud       0.07
zep        0.06
maxdep     0.05
Name: h2_agwp_ch4ind_h2o, dtype: object

### Add methane induced strat H2O GWP

In [102]:
df_h2_gwp.loc['strat H2O CH4ind'] = df_h2_agwp_ch4ind_h2o/agwp100_CO2
df_h2_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,nan,2.13,2.38,2.42,2.41,2.59,2.06,2.22,1.89,2.62,2.14,2.27
CH4,nan,4.78,4.82,4.80,4.56,5.56,4.33,4.63,3.97,5.57,4.57,4.42
strat H2O,nan,1.87,1.87,1.88,1.90,2.05,1.57,1.69,1.45,2.05,1.65,1.80
O3 CH4ind,nan,2.79,2.81,2.80,2.66,3.25,2.53,2.70,2.32,3.25,2.67,2.58
strat H2O CH4ind,nan,0.65,0.66,0.66,0.62,0.76,0.59,0.63,0.54,0.76,0.62,0.60
aerosol,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


### HYDROGEN AGWP100 methane induced aerosols [mW m-2 yr Tg-1]

In [103]:
df_h2_agwp_ch4ind_aerosol = df_aerosol_rf.loc['10CH4']/df_ch4_surfconc.loc['delta']*df_surf_ch4_per_h2_flux
df_h2_agwp_ch4ind_aerosol.name = 'h2_agwp_ch4ind_aerosols'


### Hydrogen GWP including aerosols

In [104]:
df_h2_gwp.loc['aerosol CH4ind'] = df_h2_agwp_ch4ind_aerosol/agwp100_CO2
df_h2_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,nan,2.13,2.38,2.42,2.41,2.59,2.06,2.22,1.89,2.62,2.14,2.27
CH4,nan,4.78,4.82,4.80,4.56,5.56,4.33,4.63,3.97,5.57,4.57,4.42
strat H2O,nan,1.87,1.87,1.88,1.90,2.05,1.57,1.69,1.45,2.05,1.65,1.80
O3 CH4ind,nan,2.79,2.81,2.80,2.66,3.25,2.53,2.70,2.32,3.25,2.67,2.58
strat H2O CH4ind,nan,0.65,0.66,0.66,0.62,0.76,0.59,0.63,0.54,0.76,0.62,0.60
aerosol,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
aerosol CH4ind,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [105]:
#Not include the aerosol GWP in the main table. Drop them here
df_h2_gwp = df_h2_gwp.drop(['aerosol','aerosol CH4ind'])
df_h2_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,nan,2.13,2.38,2.42,2.41,2.59,2.06,2.22,1.89,2.62,2.14,2.27
CH4,nan,4.78,4.82,4.80,4.56,5.56,4.33,4.63,3.97,5.57,4.57,4.42
strat H2O,nan,1.87,1.87,1.88,1.90,2.05,1.57,1.69,1.45,2.05,1.65,1.80
O3 CH4ind,nan,2.79,2.81,2.80,2.66,3.25,2.53,2.70,2.32,3.25,2.67,2.58
strat H2O CH4ind,nan,0.65,0.66,0.66,0.62,0.76,0.59,0.63,0.54,0.76,0.62,0.60


# Methane GWP

Initialize CH4 GWP

In [106]:
antmod = len(df_h2_agwp_ch4.index)
df_ch4_gwp = pd.DataFrame(np.empty([4,antmod])*np.nan,columns=df_h2_agwp_ch4.index,
                         index=['O3','CH4','strat H2O','H2'])
df_ch4_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
CH4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
strat H2O,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
H2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


### Methane AGWP100 O3 [mW m-2 yr Tg-1]

In [107]:
df_ch4_agwp_o3 = df_ozone_rf.loc['10CH4']/df_ch4_surfconc.loc['delta']*df_surf_ch4_per_ch4_flux
df_ch4_agwp_o3.name = 'ch4_agwp_o3'
df_ch4_agwp_o3

CNTR       0.82
antro01    0.82
antro1     0.82
antro10    0.82
antro100   0.82
nemo       0.82
epia       0.82
munich     0.82
usdrydep   0.82
maud       0.82
zep        0.82
maxdep     0.82
Name: ch4_agwp_o3, dtype: object

In [108]:
test = df_ozone_rf.loc['10CH4']/df_ch4_surfconc.loc['delta']*df_surf_ch4_per_ch4_flux
test

CNTR       0.82
antro01    0.82
antro1     0.82
antro10    0.82
antro100   0.82
nemo       0.82
epia       0.82
munich     0.82
usdrydep   0.82
maud       0.82
zep        0.82
maxdep     0.82
dtype: object

In [109]:
test2 = df_ozone_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
#test2 is equal test
test2

CNTR       0.82
antro01    0.82
antro1     0.82
antro10    0.82
antro100   0.82
nemo       0.82
epia       0.82
munich     0.82
usdrydep   0.82
maud       0.82
zep        0.82
maxdep     0.82
dtype: object

### Add ozone GWP

In [110]:
df_ch4_gwp.loc['O3'] =df_ch4_agwp_o3/agwp100_CO2 

### Methane AGWP100 Methane [mW m-2 yr Tg-1]

In [111]:
df_ch4_agwp =df_surf_ch4_per_ch4_flux*spec_rf_ch4
df_ch4_agwp.name = 'ch4_agwp'


### Add methane GWP

In [112]:
#Add Methane GWP:
df_ch4_gwp.loc['CH4'] =df_ch4_agwp/agwp100_CO2 

### Methane AGWP100 strat H2O [mW m-2 yr Tg-1]

In [113]:
df_ch4_agwp_h2o = df_h2o_rf.loc['10CH4']/df_ch4_surfconc.loc['delta']*df_surf_ch4_per_ch4_flux
df_ch4_agwp_h2o.name = 'ch4_agwp_h2o'


### Add Strat H2O GWP:

In [114]:
df_ch4_gwp.loc['strat H2O'] = df_ch4_agwp_h2o/agwp100_CO2

### Methane AGWP100 aerosols [mW m-2 yr Tg-1]

In [115]:
df_ch4_agwp_aerosol = df_aerosol_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_ch4_agwp_aerosol.name = 'ch4_agwp_aerosol'


### Add Aerosol GWP

In [116]:
df_ch4_gwp.loc['aerosol'] = df_ch4_agwp_aerosol/agwp100_CO2
df_ch4_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17
CH4,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7
strat H2O,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14
H2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
aerosol,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [117]:
#And drop the aerosol GWP in the main table:
df_ch4_gwp = df_ch4_gwp.drop(['aerosol'])
df_ch4_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17
CH4,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7
strat H2O,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14
H2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


### Add GWP via H2

In [118]:
#df_ch4_gwp.loc['H2'] = df_h2_flux_per_ch4_flux*df_h2_gwp.sum()


# Part III: Main results and tables

## H2 GWP 100

In [119]:
#model_dict = { 'OSLOCTM3':'OsloCTM',
#               'WACCM6-2deg':'WACCM',
#               'INCA':'INCA',
#               'GFDL-emi':'GFDL-emi',
#               'GFDL_nudge':'GFDL',
#               'UKCA':'UKCA',
#               'OSLOCTM3-emi':'OsloCTM-emi'}
#
#sorted_array = ['GFDL','INCA','OsloCTM','UKCA','WACCM','GFDL-emi','OsloCTM-emi']
#sorted_array_2 = ['GFDL','INCA','OsloCTM','UKCA','WACCM','GFDL-emi','OsloCTM-emi','Model mean']

In [120]:
#df_h2_gwp = df_h2_gwp[sorted(df_h2_gwp.columns)]
df_h2_gwp = df_h2_gwp.drop('CNTR',axis=1)
df_h2_gwp.loc['total']=df_h2_gwp.sum()

df_h2_gwp_table = df_h2_gwp.copy()

#df_h2_gwp_table.rename(model_dict,axis=1,inplace=True)
#df_h2_gwp_table = df_h2_gwp_table[sorted_array]
df_h2_gwp_table.T


,O3,CH4,strat H2O,O3 CH4ind,strat H2O CH4ind,total
antro01,2.13,4.78,1.87,2.79,0.65,12.2
antro1,2.38,4.82,1.87,2.81,0.66,12.5
antro10,2.42,4.80,1.88,2.80,0.66,12.6
antro100,2.41,4.56,1.90,2.66,0.62,12.2
nemo,2.59,5.56,2.05,3.25,0.76,14.2
epia,2.06,4.33,1.57,2.53,0.59,11.1
munich,2.22,4.63,1.69,2.70,0.63,11.9
usdrydep,1.89,3.97,1.45,2.32,0.54,10.2
maud,2.62,5.57,2.05,3.25,0.76,14.2
zep,2.14,4.57,1.65,2.67,0.62,11.7


## CH4 GWP 100

In [121]:
#df_ch4_gwp = df_ch4_gwp[sorted(df_ch4_gwp.columns)]
#df_ch4_gwp.drop('GFDL-emi',axis=1).to_csv(outputpath + 'table_ch4_gwp.csv')

df_ch4_gwp = df_ch4_gwp.drop('CNTR',axis=1)
df_ch4_gwp.to_csv(outputpath + 'table_ch4_gwp.csv')
df_ch4_gwp.loc['total']=df_ch4_gwp.sum()
df_ch4_gwp.T

,O3,CH4,strat H2O,H2,total
antro01,9.17,15.7,2.14,nan,27.0
antro1,9.17,15.7,2.14,nan,27.0
antro10,9.17,15.7,2.14,nan,27.0
antro100,9.17,15.7,2.14,nan,27.0
nemo,9.17,15.7,2.14,nan,27.0
epia,9.17,15.7,2.14,nan,27.0
munich,9.17,15.7,2.14,nan,27.0
usdrydep,9.17,15.7,2.14,nan,27.0
maud,9.17,15.7,2.14,nan,27.0
zep,9.17,15.7,2.14,nan,27.0


## Table per flux H2

In [122]:
df_per_flux_h2 = pd.concat([df_h2_flux['deltaH2'],
                            df_surf_h2_per_h2_flux,
                            df_surf_ch4_per_h2_flux,
                            df_ch4_flux_per_h2_flux,
                            df_ch4_rf_per_h2_flux,
                            #df_trop_du_ozone_per_h2_flux*1000.,
                            #df_strat_du_ozone_per_h2_flux*1000.,
                            df_ozone_rf_per_h2_flux,
                            df_h2o_rf_per_h2_flux,
                            df_aerosol_rf_per_h2_flux],axis=1, sort=False)



#Save to file:
df_per_flux_h2 = df_per_flux_h2.sort_index()
df_per_flux_h2.to_csv(outputpath + 'table_per_flux_h2.csv')

#Rename the columns:
columns_names={'deltaH2':'Flux H2 [Tg/yr]',
               'surf_h2_per_h2_flux': 'Surf. conc. H2 per flux [ppb yr/Tg]',
               'surf_ch4_per_h2_flux':'Surf. conc. CH4 per flux [ppb yr/Tg]',
               'ch4_flux_per_h2_flux':'Flux CH4/Flux H2 [Tg CH4/Tg H2]',
               'ch4_rf_per_h2_flux':'CH4 RF per flux [mW m-2 yr/ Tg]',
               'trop_du_ozone_per_h2_flux':'Trop. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'strat_du_ozone_per_h2_flux':'Strat. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'ozone_rf_per_h2_flux':'ozone RF per flux [mW m-2 yr/ Tg]',
               'h2o_rf_per_h2_flux':'Strat. H2O RF per flux [mW m-2 yr/ Tg]',
               'aerosol_rf_per_h2_flux':'Aerosol RF per flux [mW m-2 yr/ Tg]'}
#Rename column names:
df_per_flux_h2.rename(columns=dict(columns_names),inplace=True) #[df_per_flux_h2.columns])
#df_per_flux_h2.rename(model_dict,axis=0,inplace=True)
#df_per_flux_h2.loc['Model mean'] = df_per_flux_h2.drop(['GFDL-emi','OsloCTM-emi']).mean()
#df_per_flux_h2['Flux H2 [Tg/yr]'].loc['Model mean']=np.nan
#df_per_flux_h2=df_per_flux_h2.reindex(sorted_array_2)
df_per_flux_h2

,Flux H2 [Tg/yr],Surf. conc. H2 per flux [ppb yr/Tg],Surf. conc. CH4 per flux [ppb yr/Tg],Flux CH4/Flux H2 [Tg CH4/Tg H2],CH4 RF per flux [mW m-2 yr/ Tg],ozone RF per flux [mW m-2 yr/ Tg],Strat. H2O RF per flux [mW m-2 yr/ Tg],Aerosol RF per flux [mW m-2 yr/ Tg]
CNTR,0.00,nan,nan,nan,nan,nan,nan,nan
antro01,0.10,6.27,1.11,0.30,0.43,0.19,0.17,nan
antro1,0.99,6.27,1.12,0.31,0.43,0.21,0.17,nan
antro10,9.90,6.28,1.12,0.31,0.43,0.22,0.17,nan
antro100,99.1,6.35,1.06,0.29,0.41,0.22,0.17,nan
epia,0.99,6.05,1.01,0.28,0.39,0.18,0.14,nan
maud,0.99,9.24,1.29,0.35,0.50,0.23,0.18,nan
maxdep,0.99,5.48,1.03,0.28,0.40,0.20,0.16,nan
munich,0.99,6.32,1.08,0.29,0.41,0.20,0.15,nan
nemo,0.99,7.92,1.29,0.35,0.50,0.23,0.18,nan


## Table per flux CH4

In [123]:
df_per_flux_ch4 = pd.concat([df_ch4_flux.loc['deltaCH4'],
                            df_surf_ch4_per_ch4_flux,
                            df_h2_flux_per_ch4_flux,
                            #df_trop_du_ozone_per_ch4_flux*1000.,
                            #df_strat_du_ozone_per_ch4_flux*1000.,
                            df_ozone_rf_per_ch4_flux,
                            df_h2o_rf_per_ch4_flux,
                            df_aerosol_rf_per_ch4_flux],axis=1,sort=False)



               
#Save to file:
#df_per_flux_ch4 = df_per_flux_ch4.sort_index()
df_per_flux_ch4.to_csv(outputpath + 'table_per_flux_ch4.csv')

#Rename the columns:
columns_names={'deltaCH4':'Flux CH4 [Tg/yr]',
               'surf_ch4_per_ch4_flux':'Surf. conc. CH4 per flux [ppb yr/Tg]',
               'h2_flux_per_ch4_flux':'Flux H2/Flux CH4 [Tg H2/Tg CH4]',
               'trop_du_ozone_per_ch4_flux':'Trop. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'strat_du_ozone_per_ch4_flux':'Strat. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'ozone_rf_per_ch4_flux':'ozone RF per flux [mW m-2 yr/ Tg]',
               'h2o_rf_per_ch4_flux':'Strat H2O RF per flux [mW m-2 yr/ Tg]',
               'aerosol_rf_per_ch4_flux':'Aerosol RF per flux [mW m-2 yr/ Tg]'}
               
#Rename column names:

df_per_flux_ch4.rename(columns=dict(columns_names),inplace=True) 

#df_per_flux_ch4.rename(model_dict, inplace=True)

#df_per_flux_ch4.loc['Model mean'] = df_per_flux_ch4.drop(['GFDL-emi','OsloCTM-emi']).mean()

#df_per_flux_ch4 = df_per_flux_ch4.reindex(sorted_array_2)

df_per_flux_ch4        

,Flux CH4 [Tg/yr],Surf. conc. CH4 per flux [ppb yr/Tg],Flux H2/Flux CH4 [Tg H2/Tg CH4],ozone RF per flux [mW m-2 yr/ Tg],Strat H2O RF per flux [mW m-2 yr/ Tg],Aerosol RF per flux [mW m-2 yr/ Tg]
CNTR,49.7,3.65,0.02,0.82,0.19,NaN
antro01,49.7,3.65,0.02,0.82,0.19,NaN
antro1,49.7,3.65,0.02,0.82,0.19,NaN
antro10,49.7,3.65,0.02,0.82,0.19,NaN
antro100,49.7,3.65,0.02,0.82,0.19,NaN
nemo,49.7,3.65,0.02,0.82,0.19,NaN
epia,49.7,3.65,0.02,0.82,0.19,NaN
munich,49.7,3.65,0.02,0.82,0.19,NaN
usdrydep,49.7,3.65,0.02,0.82,0.19,NaN
maud,49.7,3.65,0.02,0.82,0.19,NaN


## Table per flux H2 (including changes in methane)

In [124]:
#Reread - to get the other heading.

df_per_flux_h2_combined = pd.read_csv(outputpath + 'table_per_flux_h2.csv',index_col=0)
#df_per_flux_h2_combined.rename(columns=dict(columns_names),inplace=True) 

#df_per_flux_h2_combined.rename(model_dict, inplace=True)

df_per_flux_h2_combined

,deltaH2,surf_h2_per_h2_flux,surf_ch4_per_h2_flux,ch4_flux_per_h2_flux,ch4_rf_per_h2_flux,ozone_rf_per_h2_flux,h2o_rf_per_h2_flux,aerosol_rf_per_h2_flux
CNTR,0.00,nan,nan,nan,nan,nan,nan,nan
antro01,0.10,6.27,1.11,0.30,0.43,0.19,0.17,nan
antro1,0.99,6.27,1.12,0.31,0.43,0.21,0.17,nan
antro10,9.90,6.28,1.12,0.31,0.43,0.22,0.17,nan
antro100,99.1,6.35,1.06,0.29,0.41,0.22,0.17,nan
epia,0.99,6.05,1.01,0.28,0.39,0.18,0.14,nan
maud,0.99,9.24,1.29,0.35,0.50,0.23,0.18,nan
maxdep,0.99,5.48,1.03,0.28,0.40,0.20,0.16,nan
munich,0.99,6.32,1.08,0.29,0.41,0.20,0.15,nan
nemo,0.99,7.92,1.29,0.35,0.50,0.23,0.18,nan


In [125]:
df_per_flux_ch4_add  = pd.read_csv(outputpath + 'table_per_flux_ch4.csv',index_col=0)
#df_per_flux_ch4_add.rename(model_dict, inplace=True)

df_per_flux_ch4_add

,deltaCH4,surf_ch4_per_ch4_flux,h2_flux_per_ch4_flux,ozone_rf_per_ch4_flux,h2o_rf_per_ch4_flux,aerosol_rf_per_ch4_flux
CNTR,49.7,3.65,0.02,0.82,0.19,nan
antro01,49.7,3.65,0.02,0.82,0.19,nan
antro1,49.7,3.65,0.02,0.82,0.19,nan
antro10,49.7,3.65,0.02,0.82,0.19,nan
antro100,49.7,3.65,0.02,0.82,0.19,nan
nemo,49.7,3.65,0.02,0.82,0.19,nan
epia,49.7,3.65,0.02,0.82,0.19,nan
munich,49.7,3.65,0.02,0.82,0.19,nan
usdrydep,49.7,3.65,0.02,0.82,0.19,nan
maud,49.7,3.65,0.02,0.82,0.19,nan


In [126]:
frac = df_per_flux_h2_combined['ch4_flux_per_h2_flux'] #Tg CH4/Tg H2 /Tg CH4 = 1/Tg H2

frac

CNTR        nan
antro01    0.30
antro1     0.31
antro10    0.31
antro100   0.29
epia       0.28
maud       0.35
maxdep     0.28
munich     0.29
nemo       0.35
usdrydep   0.25
zep        0.29
Name: ch4_flux_per_h2_flux, dtype: float64

In [127]:
df_per_flux_ch4_add
#Keep the following:
#deltaH2
#surf_h2_per_h2_flux keep as h2_flux_per_ch4_flux small
#surf_ch4_per_h2_flux
#ch4_flux_per_h2_flux
#ch4_rf_per_h2_flux

#add:
#trop_du_ozone_per_h2_flux
#strat_du_ozone_per_h2_flux
#ozone_rf_per_h2_flux
#h2o_rf_per_h2_flux
#aerosol_rf_per_h2_flux

#df_per_flux_h2_combined['trop_du_ozone_per_h2_flux'] = df_per_flux_h2_combined['trop_du_ozone_per_h2_flux'] + df_per_flux_ch4_add['trop_du_ozone_per_ch4_flux']*frac
#df_per_flux_h2_combined['strat_du_ozone_per_h2_flux'] = df_per_flux_h2_combined['strat_du_ozone_per_h2_flux'] + df_per_flux_ch4_add['strat_du_ozone_per_ch4_flux']*frac
df_per_flux_h2_combined['ozone_rf_per_h2_flux'] = df_per_flux_h2_combined['ozone_rf_per_h2_flux'] + df_per_flux_ch4_add['ozone_rf_per_ch4_flux']*frac
df_per_flux_h2_combined['h2o_rf_per_h2_flux'] = df_per_flux_h2_combined['h2o_rf_per_h2_flux'] + df_per_flux_ch4_add['h2o_rf_per_ch4_flux']*frac
df_per_flux_h2_combined['aerosol_rf_per_h2_flux'] = df_per_flux_h2_combined['aerosol_rf_per_h2_flux'] + df_per_flux_ch4_add['aerosol_rf_per_ch4_flux']*frac


#df_per_flux_h2_combined

#Save to file:
#df_per_flux_h2_combined = df_per_flux_h2_combined.sort_index()
df_per_flux_h2_combined.to_csv(outputpath + 'table_per_flux_h2_combined.csv')


In [128]:
#Rename the columns:
columns_names={'deltaH2':'Flux H2 [Tg/yr]',
               'surf_h2_per_h2_flux': 'Surf. conc. H2 per flux [ppb yr/Tg]',
               'surf_ch4_per_h2_flux':'Surf. conc. CH4 per flux [ppb yr/Tg]',
               'ch4_flux_per_h2_flux':'Flux CH4/Flux H2 [Tg CH4/Tg H2]',
               'ch4_rf_per_h2_flux':'CH4 RF per flux [mW m-2 yr/ Tg]',
               'trop_du_ozone_per_h2_flux':'Trop. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'strat_du_ozone_per_h2_flux':'Strat. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'ozone_rf_per_h2_flux':'ozone RF per flux [mW m-2 yr/ Tg]',
               'h2o_rf_per_h2_flux':'Strat. H2O RF per flux [mW m-2 yr/ Tg]',
               'aerosol_rf_per_h2_flux':'Aerosol RF per flux [mW m-2 yr/ Tg]'}
#Rename column names:
df_per_flux_h2_combined.rename(columns=dict(columns_names),inplace=True) #[df_per_flux_h2.columns])
#df_per_flux_h2_combined.rename(model_dict,axis=0,inplace=True)
#df_per_flux_h2_combined.loc['Model mean'] = df_per_flux_h2_combined.drop(['GFDL-emi','OsloCTM-emi']).mean()
#df_per_flux_h2_combined['Flux H2 [Tg/yr]'].loc['Model mean']=np.nan
#df_per_flux_h2_combined=df_per_flux_h2_combined.reindex(sorted_array_2)
df_per_flux_h2_combined

,Flux H2 [Tg/yr],Surf. conc. H2 per flux [ppb yr/Tg],Surf. conc. CH4 per flux [ppb yr/Tg],Flux CH4/Flux H2 [Tg CH4/Tg H2],CH4 RF per flux [mW m-2 yr/ Tg],ozone RF per flux [mW m-2 yr/ Tg],Strat. H2O RF per flux [mW m-2 yr/ Tg],Aerosol RF per flux [mW m-2 yr/ Tg]
CNTR,0.00,nan,nan,nan,nan,nan,nan,nan
antro01,0.10,6.27,1.11,0.30,0.43,0.44,0.23,nan
antro1,0.99,6.27,1.12,0.31,0.43,0.47,0.23,nan
antro10,9.90,6.28,1.12,0.31,0.43,0.47,0.23,nan
antro100,99.1,6.35,1.06,0.29,0.41,0.45,0.23,nan
epia,0.99,6.05,1.01,0.28,0.39,0.41,0.19,nan
maud,0.99,9.24,1.29,0.35,0.50,0.53,0.25,nan
maxdep,0.99,5.48,1.03,0.28,0.40,0.43,0.21,nan
munich,0.99,6.32,1.08,0.29,0.41,0.44,0.21,nan
nemo,0.99,7.92,1.29,0.35,0.50,0.52,0.25,nan


## H2 budget table

In [129]:
df_h2_burden


,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,205,205,207,226,423,207,207,207,206,207,207,207
deltaH2,0.00,0.22,2.15,21.5,218,2.53,1.93,2.05,1.76,2.56,2.07,1.97


In [130]:
df_budget_h2 = pd.concat([df_h2_burden.iloc[0],
                          df_h2_atmloss,
                          df_h2_atmprod,
                          df_h2_drydep,
                          df_h2_estemis, 
                          df_h2_atm_lifetime,
                          df_h2_soil_sink_lifetime,
                          df_h2_lifetime
                         ],axis=1)

df_budget_h2.columns = ['H2 burden [Tg]',
                        'H2 atm loss [Tg/yr]',
                        'H2 atm prod [Tg/yr]',
                        'H2 soil sink [Tg/yr]',
                        'H2 estimated emissions [Tg/yr]',
                        'H2 atm lifetime [yrs]',
                        'H2 soil sink lifetime [yrs]',
                        'H2 total lifetime [yrs]',]
df_budget_h2 = df_budget_h2.sort_index()
df_budget_h2.to_csv(outputpath + 'table_budget_h2.csv')
df_budget_h2



,H2 burden [Tg],H2 atm loss [Tg/yr],H2 atm prod [Tg/yr],H2 soil sink [Tg/yr],H2 estimated emissions [Tg/yr],H2 atm lifetime [yrs],H2 soil sink lifetime [yrs],H2 total lifetime [yrs]
CNTR,205,29.2,55.8,58.0,31.4,7.02,3.53,2.35
antro01,205,29.2,55.8,58.1,31.5,7.02,3.53,2.35
antro1,207,29.5,55.8,58.7,32.4,7.02,3.52,2.34
antro10,226,32.1,55.7,65.0,41.4,7.04,3.48,2.33
antro100,423,58.0,54.9,128,131,7.28,3.30,2.27
epia,207,29.4,55.8,58.7,32.4,7.02,3.52,2.34
maud,207,29.5,55.8,58.7,32.4,7.02,3.53,2.35
maxdep,207,29.4,55.8,58.7,32.4,7.02,3.52,2.34
munich,207,29.5,55.8,58.7,32.4,7.02,3.52,2.34
nemo,207,29.5,55.8,58.7,32.4,7.02,3.53,2.35


## CH4 budget table

In [131]:
df_ch4_loss_other_strat_conc.loc['CTRL']

20.729217388088895

In [132]:
budget_ch4 = np.array([df_ch4_burden_conc.loc['CTRL'],
                           df_ch4_surfconc_conc.loc['CTRL'],
                           df_ch4_loss_conc.loc['CTRL'],
                           df_ch4_loss_other_strat_conc.loc['CTRL'],
                           df_ch4_loss_soil_conc.loc['CTRL'],
                           df_ch4_loss_conc.loc['CTRL']+df_ch4_loss_other_strat_conc.loc['CTRL']+df_ch4_loss_soil_conc.loc['CTRL'],
                           df_ch4_lifetime_conc.loc['CTRL'],
                           df_ch4_tot_lifetime_conc.loc['CTRL']
                           ])

budget_ch4_columns = ['CH4 burden [Tg]','CH4 surface conc. [ppbv]',
                         'CH4 chem loss OH [Tg/yr]', 'CH4 chem loss other strat [Tg/yr]','CH4 loss soil [Tg/yr]',
                         'CH4 total loss [Tg/yr]',
                         'CH4 lifetime due to OH (whole atmosphere) [yrs]','Total CH4 lifetime [yrs]'] 

df_budget_ch4 = pd.DataFrame(data=budget_ch4,index=budget_ch4_columns,columns=['CTRL'])


In [133]:
df_budget_ch4.T.to_csv(outputpath + 'table_budget_ch4.csv')

df_budget_ch4.T

,CH4 burden [Tg],CH4 surface conc. [ppbv],CH4 chem loss OH [Tg/yr],CH4 chem loss other strat [Tg/yr],CH4 loss soil [Tg/yr],CH4 total loss [Tg/yr],CH4 lifetime due to OH (whole atmosphere) [yrs],Total CH4 lifetime [yrs]
CTRL,"4,975","1,813",674,20.7,31.1,726,7.38,6.85


In [134]:
#Write AGWP values to file
df_h2_agwp  = pd.concat([df_h2_agwp_ch4,
                         df_h2_agwp_o3,
                         df_h2_agwp_h2o,
                         df_h2_agwp_ch4ind_o3,
                         df_h2_agwp_ch4ind_h2o],axis=1,sort=False)

df_h2_agwp.to_csv(outputpath + 'table_h2_agwp.csv') 
df_h2_agwp

,h2_agwp_ch4,h2_agwp_o3,0,h2_agwp_ch4ind_o3,h2_agwp_ch4ind_h2o
CNTR,nan,nan,nan,nan,NaN
antro01,0.43,0.19,0.17,0.25,0.06
antro1,0.43,0.21,0.17,0.25,0.06
antro10,0.43,0.22,0.17,0.25,0.06
antro100,0.41,0.22,0.17,0.24,0.06
nemo,0.50,0.23,0.18,0.29,0.07
epia,0.39,0.18,0.14,0.23,0.05
munich,0.41,0.20,0.15,0.24,0.06
usdrydep,0.36,0.17,0.13,0.21,0.05
maud,0.50,0.23,0.18,0.29,0.07


# Appendix:

## Methane feedback factor:

### Atmospheric mass conversion CH4  [Tg/ppb] (from perturbations)

In [135]:
df_ch4_burden_per_conc  = df_ch4_burden_conc.loc['deltaCH4']/df_ch4_surfconc_conc.loc['deltaCH4']
#df_ch4_burden_per_conc.name = 'ch4_burden_per_conc'
df_ch4_burden_per_conc

2.7494761488687924

### Increase per unit flux w/o feedback = integrated decay [ppb yr/Tg]

In [136]:
df_w_o_feedback =df_ch4_tot_lifetime_conc.loc['CTRL']/df_ch4_burden_per_conc #Lifetime [yr] / [Tg/ppb] 
#df_w_o_feedback.name = 'increase_w_o_feedback'
df_w_o_feedback_to_file = pd.DataFrame(data=[df_w_o_feedback],index=['OSLOCTM3'])
df_w_o_feedback_to_file.to_csv(outputpath + 'increase_w_o_feedback.csv')
df_w_o_feedback

2.4923974591529294

### Feedback factor: increase CH4 with feedback/ increase CH4 without feedback

In [137]:
df_feedback_factor_ch4 = df_surf_ch4_per_ch4_flux_conc/df_w_o_feedback
#df_feedback_factor_ch4.name = 'feedback_factor_ch4'
df_feedback_to_file = pd.DataFrame(data=[df_feedback_factor_ch4],index=['CTRL'])
df_feedback_to_file.to_csv(outputpath + 'feedback_factor_ch4.csv')
df_feedback_factor_ch4

1.4645748747886003

Split the CH4 GWP into direct and indirect based on the feedback factor.

In [138]:
feedback_factor = df_feedback_factor_ch4
feedback_frac = 1.0 - (1.0/feedback_factor)
#feedback_frac.name = 'feedback_frac'
feedback_frac

0.3172080053986027

In [139]:
#Save to file:

df_h2_gwp.loc['CH4dir'] = df_h2_gwp.loc['CH4']*(1.0-feedback_frac)
df_h2_gwp.loc['CH4indir'] = df_h2_gwp.loc['CH4']*feedback_frac

#df_h2_gwp['GFDL-emi'].loc['CH4dir'] = df_h2_gwp['GFDL-emi'].loc['CH4']
#df_h2_gwp['OSLOCTM3-emi'].loc['CH4dir'] = df_h2_gwp['OSLOCTM3-emi'].loc['CH4']
df_h2_gwp = df_h2_gwp.drop(['total','CH4'])
df_h2_gwp.to_csv(outputpath + 'table_h2_gwp.csv')

df_h2_gwp

,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,2.13,2.38,2.42,2.41,2.59,2.06,2.22,1.89,2.62,2.14,2.27
strat H2O,1.87,1.87,1.88,1.90,2.05,1.57,1.69,1.45,2.05,1.65,1.80
O3 CH4ind,2.79,2.81,2.80,2.66,3.25,2.53,2.70,2.32,3.25,2.67,2.58
strat H2O CH4ind,0.65,0.66,0.66,0.62,0.76,0.59,0.63,0.54,0.76,0.62,0.60
CH4dir,3.26,3.29,3.28,3.11,3.80,2.95,3.16,2.71,3.80,3.12,3.02
CH4indir,1.52,1.53,1.52,1.45,1.76,1.37,1.47,1.26,1.77,1.45,1.40


In [140]:
df_feedback_factor_ch4

1.4645748747886003

In [141]:
#Alternative feedback calc:
#ss the sensitivity of the lifetime to the burden.

#ss = 1 – dln[<k All><H2>]/dln[<H2>] 
#= 1 – dln[<k All>]/dln[<H2>] – dln[<H2>]/dln[<H2>] 
#= – dln[<k All>]/dln[<H2>] 
#=  +dln[1/<k All>] / dln[<H2>] 

#<k All> = <k OH> + <kstrat OH> + <k soil> Are the inverse liftime-
k_all_inv = df_ch4_tot_lifetime_conc

a = np.log(k_all_inv.loc['10CH4'])-np.log(k_all_inv.loc['CTRL'])

b = np.log(df_ch4_burden_conc.loc['10CH4'])-np.log(df_ch4_burden_conc.loc['CTRL'])
#print(b)
ss = a/b
ff = 1/(1-ss)
print(ff)

1.4429794488833552


In [142]:
ss = (np.log(df_ch4_tot_lifetime_conc.loc['10CH4']/df_ch4_tot_lifetime_conc.loc['CTRL']))/np.log(df_ch4_burden_conc.loc['10CH4']/df_ch4_burden_conc.loc['CTRL'])
ff=1/(1-ss)
print(ff)

1.4429794488833612


## Hydrogen feedback factor:

### Atmospheric mass conversion H2  [Tg/ppb] (from perturbations)

In [143]:
df_h2_burden_per_conc  = df_h2_burden.loc['deltaH2']/df_h2_surfconc.loc['delta']
df_h2_burden_per_conc.name = 'h2_burden_per_conc'
df_h2_burden_per_conc

CNTR        nan
antro01    0.35
antro1     0.35
antro10    0.35
antro100   0.35
nemo       0.32
epia       0.32
munich     0.33
usdrydep   0.34
maud       0.28
zep        0.31
maxdep     0.36
Name: h2_burden_per_conc, dtype: float64

### Increase per unit flux w/o feedback = integrated decay [ppb yr/Tg]

In [144]:
df_w_o_feedback_h2 =df_h2_lifetime.loc['CNTR']/df_h2_burden_per_conc #Lifetime [yr] / [Tg/ppb] 
df_w_o_feedback_h2

CNTR        nan
antro01    6.78
antro1     6.78
antro10    6.78
antro100   6.78
nemo       7.27
epia       7.30
munich     7.13
usdrydep   6.88
maud       8.39
zep        7.52
maxdep     6.45
Name: h2_burden_per_conc, dtype: float64

### Feedback factor: increase H2 with feedback/ increase H2 without feedback

If you take the control lifetime from the budget terms and divide it into the lifetime of the perturbation (the added burden from the 10% increase divided by the change in chemical flux from that perturbation.) You should get the feed back factor.

In [145]:
df_feedback_factor_h2 = df_surf_h2_per_h2_flux/df_w_o_feedback_h2
df_feedback_factor_h2.name = 'feedback_factor_h2'
df_feedback_factor_h2.to_csv(outputpath + 'feedback_factor_h2.csv')
df_feedback_factor_h2
#Fabien wrote in the paper about feedback factor less than 1.

CNTR        nan
antro01    0.92
antro1     0.92
antro10    0.93
antro100   0.94
nemo       1.09
epia       0.83
munich     0.89
usdrydep   0.76
maud       1.10
zep        0.89
maxdep     0.85
Name: feedback_factor_h2, dtype: float64

### Change in lifetime per flux

In [146]:
df_ch4_lifetime.loc['deltaH2'] = df_ch4_lifetime.iloc[0]-df_ch4_lifetime['CNTR'].iloc[0]
df_ch4_lifetime.loc['deltaCH4'] = df_ch4_lifetime_conc.loc['10CH4']-df_ch4_lifetime_conc.loc['CTRL']
df_ch4_lifetime

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,7.39,7.39,7.40,7.43,7.73,7.40,7.40,7.40,7.40,7.40,7.40,7.40
deltaH2,0.00,0.00,0.00,0.03,0.33,0.00,0.00,0.00,0.00,0.00,0.00,0.00
deltaCH4,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24


In [147]:
#Direct (changes in methane lifetime per h2 flux [days per Tg H2])
df_ch4_lifetime_per_h2_flux =  df_ch4_lifetime.loc['deltaH2']/df_h2_flux['deltaH2']
df_ch4_lifetime_per_h2_flux*365.0 #Days

CNTR        nan
antro01    1.22
antro1     1.23
antro10    1.23
antro100   1.22
nemo       1.42
epia       1.11
munich     1.19
usdrydep   1.02
maud       1.42
zep        1.17
maxdep     1.13
Name: deltaH2, dtype: float64

In [148]:
#Indirect (changes in methane lifetime per h2 flux [days per Tg H2] due to changes in methane):
df_ch4_lifetime_per_ch4_flux =  df_ch4_lifetime.loc['deltaCH4']/df_ch4_flux.loc['deltaCH4']
df_ch4_lifetime_per_ch4_flux*365.0*df_ch4_flux_per_h2_flux


CNTR        nan
antro01    0.53
antro1     0.53
antro10    0.53
antro100   0.51
nemo       0.62
epia       0.48
munich     0.51
usdrydep   0.44
maud       0.62
zep        0.51
maxdep     0.49
dtype: float64

### Check that delta flux equal delta burden divided by lifetime including the feedback effect

In [149]:
df_feedback_factor_h2

CNTR        nan
antro01    0.92
antro1     0.92
antro10    0.93
antro100   0.94
nemo       1.09
epia       0.83
munich     0.89
usdrydep   0.76
maud       1.10
zep        0.89
maxdep     0.85
Name: feedback_factor_h2, dtype: float64

In [150]:
df_h2_burden

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,205,205,207,226,423,207,207,207,206,207,207,207
deltaH2,0.00,0.22,2.15,21.5,218,2.53,1.93,2.05,1.76,2.56,2.07,1.97


In [151]:
ss = (np.log(df_h2_lifetime/df_h2_lifetime.loc['CNTR']))/np.log(df_h2_burden.iloc[0]/df_h2_burden['CNTR'].iloc[0])
ff=1/(1-ss)
print(ff)

CNTR        nan
antro01    0.92
antro1     0.93
antro10    0.93
antro100   0.96
nemo       1.09
epia       0.83
munich     0.89
usdrydep   0.76
maud       1.10
zep        0.89
maxdep     0.85
dtype: float64


In [152]:
#Delta burden divided by tau*ff
test = df_h2_burden.loc['deltaH2']/(df_h2_lifetime.loc['CNTR']*df_feedback_factor_h2)
print(test)
print(df_h2_flux['deltaH2'])


CNTR        nan
antro01    0.10
antro1     0.99
antro10    9.90
antro100   99.1
nemo       0.99
epia       0.99
munich     0.99
usdrydep   0.99
maud       0.99
zep        0.99
maxdep     0.99
dtype: float64
CNTR       0.00
antro01    0.10
antro1     0.99
antro10    9.90
antro100   99.1
nemo       0.99
epia       0.99
munich     0.99
usdrydep   0.99
maud       0.99
zep        0.99
maxdep     0.99
Name: deltaH2, dtype: float64


In [153]:
(df_ch4_tot_lifetime.iloc[0]['CNTR']*df_feedback_factor_ch4)

10.055132897458343

In [154]:
test = df_ch4_burden.loc['deltaCH4']/(df_ch4_tot_lifetime.iloc[0]['CNTR']*df_feedback_factor_ch4)
print(test)
print(df_ch4_flux.loc['deltaCH4'])

CNTR       0.00
antro01    0.00
antro1     0.00
antro10    0.00
antro100   0.04
nemo       0.00
epia       0.00
munich     0.00
usdrydep   0.00
maud       0.00
zep        0.00
maxdep     0.00
Name: deltaCH4, dtype: float64
CNTR       49.7
antro01    49.7
antro1     49.7
antro10    49.7
antro100   49.7
nemo       49.7
epia       49.7
munich     49.7
usdrydep   49.7
maud       49.7
zep        49.7
maxdep     49.7
Name: deltaCH4, dtype: float64


## Feedback factor summary

The feedback factor calculation employed above is equivalent with a division between the perturbation lifetime and the total lifetime, where the perturbation lifetime is defined as the burden change diveded by the flux change in the perturbed simulation relative to the control simulation

In [155]:
print(df_feedback_factor_ch4)
test_ff =  (df_ch4_burden_conc.loc['deltaCH4']/df_ch4_flux_conc.loc['deltaCH4'])/df_ch4_tot_lifetime_conc.loc['CTRL']

test_ff

1.4645748747886003


1.4645748747886005

In [156]:
print(df_feedback_factor_h2)
test_ff =  (df_h2_burden.loc['deltaH2']/df_h2_flux['deltaH2'])/df_h2_lifetime.loc['CNTR']

test_ff



CNTR        nan
antro01    0.92
antro1     0.92
antro10    0.93
antro100   0.94
nemo       1.09
epia       0.83
munich     0.89
usdrydep   0.76
maud       1.10
zep        0.89
maxdep     0.85
Name: feedback_factor_h2, dtype: float64


CNTR        nan
antro01    0.92
antro1     0.92
antro10    0.93
antro100   0.94
nemo       1.09
epia       0.83
munich     0.89
usdrydep   0.76
maud       1.10
zep        0.89
maxdep     0.85
Name: deltaH2, dtype: float64

## Calculating values needed for uncertainty calculations

Calculating various means and spreads needed to perform uncertainty calculations. The calculations are in practice done in another notebook, GWP_different_time_horixons.ipynb, using these values.

tau = np.mean(df_h2_lifetime.drop(labels=['GFDL-emi', 'OSLOCTM3-emi'], axis=1).loc[['CTRL']].values*df_feedback_factor_h2.drop(labels=['GFDL-emi', 'OSLOCTM3-emi']).values)
gfdl_scaled_burden = df_h2_burden.loc['deltaH2']['GFDL_nudge']/4
burden_for_uncertainties = np.append(df_h2_burden.drop(labels=['GFDL-emi', 'OSLOCTM3-emi', 'GFDL_nudge'], axis=1).loc[['deltaH2']].values,gfdl_scaled_burden)
sigma_burden = np.std(burden_for_uncertainties)
sigma_atmloss = np.std(df_h2_atmloss.drop(labels=['GFDL-emi', 'OSLOCTM3-emi'], axis=1).loc[['CTRL']].values)
sigma_delta_flux_CH4_dir = np.std(df_per_flux_ch4.drop(labels=['GFDL-emi', 'OsloCTM-emi', 'Model mean'], axis=0).mean(axis = 1).values/df_feedback_factor_ch4.drop(labels=['GFDL-emi', 'OSLOCTM3-emi']).values)

sigma_h2_ff = np.std(df_feedback_factor_h2.drop(labels=['GFDL-emi', 'OSLOCTM3-emi']))

delta_burden = np.mean(burden_for_uncertainties)
loss = np.mean(df_h2_loss.drop(labels=['GFDL-emi', 'OSLOCTM3-emi'], axis=1).loc[['CTRL']].values)
delta_flux_CH4 = df_per_flux_ch4.drop(labels=['GFDL-emi', 'OsloCTM-emi'], axis=0)['Surf. conc. CH4 per flux [ppb yr/Tg]'].mean()
ff_ch4 = df_feedback_factor_ch4.drop(labels=['GFDL-emi', 'OSLOCTM3-emi']).mean()
ff_h2 = df_feedback_factor_h2.drop(labels=['GFDL-emi', 'OSLOCTM3-emi']).mean()
delta_flux_CH4_dir = delta_flux_CH4*1.0/ff_ch4

df_needed_for_uncertainties = pd.DataFrame(data={"delta_burden":delta_burden, "sigma_burden":sigma_burden, "loss":loss, "sigma_atmloss":sigma_atmloss, "sigma_h2_ff":sigma_h2_ff, "ff_h2":ff_h2, "tau":tau, "ff_ch4":ff_ch4, "delta_flux_CH4":delta_flux_CH4, "sigma_delta_flux_CH4_dir":sigma_delta_flux_CH4_dir}, index = [0])
print(df_needed_for_uncertainties)
df_needed_for_uncertainties.to_csv(outputpath + 'uncertainty_input.txt')